### Scrap web data into excel and format for easy filtering.

I'm looking to pull the data from https://seafoodfromcanada.ca/directory/ and put it into a Google Sheet.

Each Profile has many details (exports to, specifies available etc) that I want included in the sheet.

Each profile should be it's own line item with all the details (categories should be like :Name, Phone number, e-mail, address, exports to, species etc)

Also, I want to be able to filter quickly to which profile suits my search. For example, on the excel sheet I want to filter for profiles that export to Asia AND Lobster specie.

In [39]:
from selenium import webdriver  # driver
from selenium.webdriver.chrome.options import Options  # setting User Agent
from selenium.common.exceptions import StaleElementReferenceException, NoSuchElementException, NoSuchWindowException
from selenium.webdriver.common.by import By # helps implicit waiting
from selenium.webdriver.support import expected_conditions as EC # this one too
from selenium.webdriver.support.ui import WebDriverWait # implicit waiting
from selenium.webdriver.common.keys import Keys
import time
import csv
import re

In [2]:
path = "C:\webdrivers\chromedriver_win32\chromedriver.exe"

In [3]:
agent = "user-agent=Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/77.0.3865.90 Safari/537.36"

In [4]:
#headless = "--headless"

In [5]:
the_options = webdriver.ChromeOptions()
the_options.add_argument(path)
#the_options.add_argument(headless)

In [6]:
driver = webdriver.Chrome(path, options=the_options)

In [7]:
url = 'https://seafoodfromcanada.ca/directory/'

In [8]:
driver.maximize_window()

In [9]:
driver.get(url)

In [10]:
next_button = driver.find_element_by_xpath("(//a[starts-with(@class, 'f-next')])[2]")

In [11]:
def get_links():
    links_list = []
    pages_done = False
    last_page = '18\nof\n18'
    while pages_done is not True:
        current_page = driver.find_element_by_xpath('(//span[starts-with(@class,"f-pagination")])').text
        links = driver.find_elements_by_xpath('(//article[starts-with(@class,"gi gi-listing")]/a)')
        for link in links:
            one_link = link.get_attribute('href')
            links_list.append(one_link)
                
        if current_page == last_page:
            pages_done = True
        
        next_button = driver.find_element_by_xpath("(//a[starts-with(@class, 'f-next')])[2]")
        next_button.click()
        time.sleep(1)

    return links_list

In [ ]:
listing-detail listing-phone

In [12]:
businesses = get_links()

In [13]:
businesses

['https://seafoodfromcanada.ca/listing/3-ts-ltd/',
 'https://seafoodfromcanada.ca/listing/a-b-seafood-specialists/',
 'https://seafoodfromcanada.ca/listing/a-l-leblanc-limited/',
 'https://seafoodfromcanada.ca/listing/al-seafoods-ltd/',
 'https://seafoodfromcanada.ca/listing/abriel-fisheries-company/',
 'https://seafoodfromcanada.ca/listing/ac-covert-limited/',
 'https://seafoodfromcanada.ca/listing/ac-live-lobster-inc/',
 'https://seafoodfromcanada.ca/listing/acadian-fish-farm-ltd-pisciculture-acadienne-ltee/',
 'https://seafoodfromcanada.ca/listing/acadian-seaplants-limited/',
 'https://seafoodfromcanada.ca/listing/acadian-sturgeon-and-caviar-inc/',
 'https://seafoodfromcanada.ca/listing/acadian-fishermens-co-op-association-ltd/',
 'https://seafoodfromcanada.ca/listing/acr-world-wide-ltd/',
 'https://seafoodfromcanada.ca/listing/adrice-cormier-ltd/',
 'https://seafoodfromcanada.ca/listing/age-lobster-inc/',
 'https://seafoodfromcanada.ca/listing/alder-point-marine-services-ltd/',
 'h

In [14]:
len(businesses)

355

In [25]:
business_name = driver.find_element_by_xpath('//h1').text

business_name

'A B Seafood Specialists'

In [26]:
contact_person = driver.find_element_by_xpath("//div[@class='listing-contact pt3']/div").text

contact_person

'Brice Hornbrook/Bruce Hornbrook'

In [27]:
business_item = driver.find_element_by_xpath("//div[starts-with(@class, 'sm-flex')]")

In [28]:
print(business_item.text)

506-543-0669 (Brice) / 506-543-1668 (Bruce)
abseafood@nb.aibn.com
1715 Gowan Brae Drive
Bathurst NB
E2A 4X5


In [34]:
email_address = driver.find_element_by_xpath("//div[starts-with(@class, 'listing-detail listing-email')]").text

email_address

'abseafood@nb.aibn.com'

In [35]:
physical_address = driver.find_element_by_xpath("//div[starts-with(@class, 'listing-data lh2')]").text

physical_address

'1715 Gowan Brae Drive\nBathurst NB\nE2A 4X5'

In [37]:
exports_to = driver.find_element_by_xpath("//ul[starts-with(@class, 'nolist lh2 mt2')]").text

exports_to

'Asia\nEurope\nNorth America'

In [38]:
species_available = driver.find_element_by_xpath("//ul[starts-with(@class, 'listing-species')]").text

species_available

'Crab\nCrab, Rock\nCrab, Snow (Queen)\nHalibut, Atlantic\nHalibut, Greenland (Turbot)\nHerring, Atlantic\nLobster\nMackerel, Atlantic\nMonkfish'

In [68]:
def scrape():
    
    table = []
    headers = ['Business Name', 'Contact Person',
               'Email Address', 'Physical Address',
               'Exports To', 'Species Available']
    table.append(headers)
    
    for business in businesses[0:20]:
        driver.get(business)
        business_name = driver.find_element_by_xpath('//h1').text
        contact_person = driver.find_element_by_xpath("//div[@class='listing-contact pt3']/div").text
        email_address = driver.find_element_by_xpath("//div[starts-with(@class, 'listing-detail listing-email')]").text
        physical_address = driver.find_element_by_xpath("//div[starts-with(@class, 'listing-data lh2')]").text
        exports_to = driver.find_element_by_xpath("//ul[starts-with(@class, 'nolist lh2 mt2')]").text
        species_available = driver.find_element_by_xpath("//ul[starts-with(@class, 'listing-species')]").text
        
        record = [business_name, contact_person,
                 email_address, physical_address,
                 exports_to, species_available]
        table.append(record)
    
    return table

In [69]:
data = scrape()

In [71]:
len(data)

21

In [72]:
data

[['Business Name',
  'Contact Person',
  'Email Address',
  'Physical Address',
  'Exports To',
  'Species Available'],
 ["3 T's Ltd.",
  'Todd Young',
  'todd3ts@hotmail.com',
  'P. O. Box 71\nWoody Point NL\nA0K 1P0',
  'Asia\nEurope\nNorth America',
  'Capelin\nCod, Atlantic\nCrab\nCrab, Snow (Queen)\nHalibut, Atlantic\nHalibut, Greenland (Turbot)\nHerring, Atlantic\nLobster\nMackerel, Atlantic'],
 ['A B Seafood Specialists',
  'Brice Hornbrook/Bruce Hornbrook',
  'abseafood@nb.aibn.com',
  '1715 Gowan Brae Drive\nBathurst NB\nE2A 4X5',
  'Asia\nEurope\nNorth America',
  'Crab\nCrab, Rock\nCrab, Snow (Queen)\nHalibut, Atlantic\nHalibut, Greenland (Turbot)\nHerring, Atlantic\nLobster\nMackerel, Atlantic\nMonkfish'],
 ['A.L. LeBlanc Limited',
  'Lisa Doucette',
  'lisadoucette@eastlink.ca',
  '2473 Hwy 334\nWedgeport NS\nB0W 3P0',
  'North America',
  'Herring, Atlantic\nLobster'],
 ['A&L Seafoods Ltd.',
  'Joe Anthony',
  'joeanthony@ns.sympatico.ca',
  '20 Minto Street\nLouisbourg N